In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scs

import pyet
import pyeto
import spei

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

monthly_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])

monthly_PET = pd.read_sql(sql='SELECT * FROM pet_monthly', con=conn)
monthly_PET['Date'] = pd.to_datetime(monthly_PET['Date'])

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [4]:
monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Station_ID           5136 non-null   object        
 1   Date                 5136 non-null   datetime64[ns]
 2   Temperature_Maximum  5136 non-null   float64       
 3   Temperature_Minimum  5136 non-null   float64       
 4   Temperature          5136 non-null   float64       
 5   Precipitation        5136 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 240.9+ KB


In [5]:
monthly_PET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Station_ID      5136 non-null   object        
 1   Date            5136 non-null   datetime64[ns]
 2   PET_Hargreaves  5136 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 120.5+ KB


In [6]:
data_si = monthly_data\
    .merge(monthly_PET, on=['Station_ID', 'Date'], how='outer')\
    .set_index(['Date'])[['Station_ID', 'Precipitation', 'PET_Hargreaves']]

data_si['PE_Hargreaves'] = data_si['Precipitation'] - data_si['PET_Hargreaves']


timescale = [1, 3, 6, 9, 12, 15, 18, 21, 24]
i = 1

for ts in timescale:
    df_spi = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spi(
                series=x.Precipitation,
                dist=scs.gamma,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()
    df_spi = df_spi.melt(id_vars="Station_ID", value_name=f'SPI_{ts}')
    if i == 1:
        results = df_spi.copy()
    else:
        results = results.merge(df_spi, on=['Station_ID', 'Date'], how='outer')
    
    df_spei = data_si\
        .groupby(by='Station_ID')\
        .apply(
            lambda x: spei.spei(
                series=x.PE_Hargreaves,
                dist=scs.fisk,
                prob_zero=True,
                timescale=ts
            ),
            include_groups=False
        )\
        .reset_index()
    df_spei = df_spei.melt(id_vars="Station_ID", value_name=f'SPEI_{ts}')
    results = results.merge(df_spei, on=['Station_ID', 'Date'], how='outer')

    i += 1


results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Station_ID  5136 non-null   object 
 1   Date        5136 non-null   object 
 2   SPI_1       5136 non-null   float64
 3   SPEI_1      5136 non-null   float64
 4   SPI_3       5088 non-null   float64
 5   SPEI_3      5088 non-null   float64
 6   SPI_6       5016 non-null   float64
 7   SPEI_6      5016 non-null   float64
 8   SPI_9       4944 non-null   float64
 9   SPEI_9      4944 non-null   float64
 10  SPI_12      4872 non-null   float64
 11  SPEI_12     4872 non-null   float64
 12  SPI_15      4800 non-null   float64
 13  SPEI_15     4800 non-null   float64
 14  SPI_18      4728 non-null   float64
 15  SPEI_18     4728 non-null   float64
 16  SPI_21      4656 non-null   float64
 17  SPEI_21     4656 non-null   float64
 18  SPI_24      4584 non-null   float64
 19  SPEI_24     4584 non-null  

In [9]:
results['Date'] = pd.to_datetime(results['Date'])
results

,Station_ID,Date,SPI_1,SPEI_1,SPI_3,SPEI_3,SPI_6,SPEI_6,SPI_9,SPEI_9,SPI_12,SPEI_12,SPI_15,SPEI_15,SPI_18,SPEI_18,SPI_21,SPEI_21,SPI_24,SPEI_24
0,40709,2006-09-30,0.577097,0.585325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40709,2006-10-31,0.181880,0.223314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40709,2006-11-30,-0.404770,-0.518566,0.021855,0.051675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40709,2006-12-31,-0.302712,-0.376520,0.104363,-0.004688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40709,2007-01-31,-1.251349,-1.597867,-1.009109,-1.316149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,99361,2024-02-29,0.884308,0.853012,-0.030903,-0.400525,0.026118,-0.220429,0.322690,-0.119020,-0.213260,-0.759940,-0.720390,-1.047768,-1.240657,-1.509941,-1.502184,-1.734198,-6.775148,-1.808627
5132,99361,2024-03-31,-1.908553,-1.488668,-0.348478,-0.622804,-0.375839,-0.885969,-0.559686,-0.867645,-0.486427,-0.927527,-0.910078,-1.326056,-1.499701,-1.617356,-1.588365,-1.762774,-1.808909,-1.889524
5133,99361,2024-04-30,0.487989,0.243740,0.624733,-0.169329,-0.402727,-0.798317,-0.349255,-0.700619,-0.155613,-0.613632,-0.572782,-1.045780,-1.249566,-1.440163,-1.438278,-1.657451,-1.679249,-1.840490
5134,99361,2024-05-31,0.967801,1.020844,-0.394997,-0.504717,-0.160645,-0.476082,-0.082741,-0.402092,0.098535,-0.264682,-0.306787,-0.827431,-0.784768,-1.102732,-1.227741,-1.520228,-0.215755,-1.822163


In [10]:
conn = sqlite3.connect(DATABASE_PATH)

results.to_sql('si_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()